## Ranking Based Approach for Individual Fairness in GNNs 

The main objective of this paper is to ensure individual fairness within graph neural networks while also preserving the performance of GNNs. Individual fairness refers to the notion of similar outcomes for individuals who share similar characteristics. To accomplish this, the research conducts experiments on two specific tasks: Node Classification and Link Prediction. These experiments are carried out using three different datasets, incorporating two backbone models, as well as two distinct similarity metrics and ranking metrics for each task.

### What is Individual Fairness? 

Firstly, what does individual fairness mean for Graph Neural Networks (GNNs)? We can discuss two fairness paradigms from this standpoint, namely individual and group fairness. Group fairness pertains to whether the predictions of models are similar for similar members of different subgroups (e.g. Hispanic, Asian). On the other hand, individual fairness focuses on individuals rather than groups. In this context, models are expected to provide similar outcomes for similar individuals. In this paper, the authors prefer to adopt a ranking-based approach instead of a distance-based approach for the sake of fairness.

In the distance-based approach, the main objective of fairness optimization is to ensure that the distances between two nodes in the input and output spaces remain as similar as possible. On the other hand, the ranking-based approach aims to maximize the similarity between rankings, which is determined by the distances between one node and other nodes in both the input and output spaces. In the ranking-based approach, each node is associated with two ranking lists. The first ranking list (S_G) is generated from the input space, while the second list (S_Y) is obtained from the model's prediction. The node with the closest distance to the target node holds the first rank in the ranking list. The ultimate goal, in order to achieve individual fairness, is to minimize the dissimilarity between these two ranking lists.

### Node Similarity and Ranking Similarity

So, we need to calculate similarity matrices for the input and output spaces. Additionally, we need to employ a metric for comparing these similarity matrices. The authors of this paper utilized two similarity metrics to assess the similarities between two nodes. They employed cosine similarity to evaluate feature-based similarities, while Jaccard similarity was used for structural comparisons. In order to compare two distinct rankings, they adopted Expected Reciprocal Rank (ERR) and Cumulative Gain-based approach (NDCG). Furthermore, they focused on the first k elements within the ranking lists to mitigate algorithmic complexity.

### Backbone Models and Their Performances 

REDRESS is a modular framework consisting of two main modules. The first module aims to maximize the usefulness of the backbone model, while the second module focuses on achieving fairness. At the end, the loss function incorporates two terms for these modules. Additionally, the contribution of the fairness module can be adjusted by setting the hyperparameter gamma (𝛾). In this paper, Graph Convolutional Network (GCN) and Simplifying Graph Convolutional Network (SGC) are employed for the Node Classification task, while GCN and Variational Graph Auto-Encoders (GAE) are utilized for the Link Prediction task.

To assess the performance of the backbone models, accuracy (ACC) is employed for Node Classification, and the area under the receiver operating characteristic curve (AUC) is used for Link Prediction.

![scale=0.5](tables/overview.png)

### Datasets

We utilize three datasets for each downstream task. For the Node Classification task, we employ one citation network (ACM) and two co-authorship networks (Co-author-CS and Co-author-Phy). On the other hand, for the Link Prediction task, we utilize three social networks (BlogCatalog, Flickr, and Facebook).

In the citation network, each node represents a paper, and the edges between nodes indicate the citation relationships between papers. In the co-authorship networks, each node represents an author, and the edges represent collaboration relationships between them.

To calculate node attributes, we utilize the bag-of-words model, which involves analyzing the abstracts of the papers. For more detailed statistics regarding the datasets, please refer to the table below.

![scale=0.5](tables/dataset.png)

### Reproducibility and Our Results

All datasets are accessible to the public, and the authors of the paper have already shared their repository. Additionally, they have provided detailed explanations for the sake of reproducibility. As a result, I have obtained similar scores to those mentioned in the paper, and I have not noticed any inconsistencies. I have successfully replicated some of the results. 

Instead of rerunning all 48 experiments, which would take approximately two days to complete due to running 24 trials for each task, I decided to include a subset of experiments for reporting regenerated results. The selection of which experiments to reproduce for reporting was based on factors such as diversity, dataset sizes, and the time required to complete the experiments.

By considering diversity, I aimed to include experiments that cover a wide range of scenarios, ensuring a representative sample of the overall experimental space. This approach allows for capturing different aspects and variations present in the tasks.

Additionally, dataset sizes played a role in the decision-making process. I considered the importance of including experiments that cover different dataset scales or distributions. By including a mix of large and small datasets, I could assess the generalizability and performance of the models across various data sizes.

Lastly, the time required for experiments to converge and produce results was taken into account. Since running all 48 experiments would take a considerable amount of time, I prioritized selecting experiments that yielded results relatively quickly. This approach allowed for a more timely analysis and reporting of the findings.

By considering these factors, I ensured that the subset of experiments chosen for reproduction and reporting would provide meaningful insights while optimizing the use of time and resources.

In [1]:
import pandas as pd
import subprocess
from collections import defaultdict
import json 
import re

In [2]:
node_classification_setup = {
    "dataset": ["ACM", "coauthor-cs", "coauthor-phy"],
    "model": ["SGC", "GCN"],
    "node_similarity": ["feature", "structural"],
    "ranking_similarity": ["NDCG", "ERR"]
}

link_prediction_setup = {
    "dataset": ["BlogCatalog", "facebook", "Flickr"],
    "model": ["GCN", "GAE"],
    "node_similarity": ["feature", "structural"],
    "ranking_similarity": ["NDCG", "ERR"]
}

In [3]:
def log_parser(log, fair_pattern, utility_pattern):
    fair = re.findall(fair_pattern, log[-1])
    utility = re.findall(utility_pattern, log[-1])
    if fair and utility:
        return log[:-1] + [round(float(fair[0]), 2), round(float(utility[0]), 2)]
    else:
        return log[:-1] + [None, None]


In [4]:
with open("node classification_results.json") as f:
    nc_result = json.load(f)

parsed_nc_result = [
    log_parser(
        experiment,
        "@k =  ([0-9.]+)\nTest set results",
        "accuracy= ([0-9.]+)\n"
    )
    for experiment in nc_result
]

In [5]:
with open("link prediction_results.json") as f:
    lp_result = json.load(f)

parsed_lp_result = [
    log_parser(
        experiment,
        "fair_after\[([0-9.]+)\]",
        "auc_after\[([0-9.]+)\]",
    )
    for experiment in (lp_result)
]


In [6]:
def get_dict_for_df(parsed_result):
    parsed_dict = defaultdict(list)
    for r in parsed_result:
        parsed_dict["Dataset"].append(r[3])
        parsed_dict["Backbone"].append(r[4])
        parsed_dict["Node Similarity"].append(r[1])
        parsed_dict["Ranking Similarity"].append(r[2])
        parsed_dict["Utility"].append(r[6])
        parsed_dict["Fairness"].append(r[5])
    return parsed_dict

#### Node Classification Reproduced Results

In [7]:
pd.DataFrame(get_dict_for_df(parsed_nc_result)).dropna()

,Dataset,Backbone,Node Similarity,Ranking Similarity,Utility,Fairness
0,ACM,SGC,feature,NDCG,0.67,0.60
1,ACM,GCN,feature,NDCG,0.70,0.31
2,coauthor-cs,SGC,feature,NDCG,0.89,0.76
3,coauthor-cs,GCN,feature,NDCG,0.92,0.47
6,ACM,SGC,feature,ERR,0.66,0.83
7,ACM,GCN,feature,ERR,0.70,0.76
8,coauthor-cs,SGC,feature,ERR,0.91,0.92
9,coauthor-cs,GCN,feature,ERR,0.92,0.79
12,ACM,SGC,structural,NDCG,0.67,0.39
13,ACM,GCN,structural,NDCG,0.68,0.25


![](tables/nc_ndcg.png)


![scale=0.5](tables/nc_err.png)


#### Link Prediction Reproduced Results

In [8]:
pd.DataFrame(get_dict_for_df(parsed_lp_result)).dropna()

,Dataset,Backbone,Node Similarity,Ranking Similarity,Utility,Fairness
2,facebook,GCN,feature,NDCG,0.96,0.29
3,facebook,GAE,feature,NDCG,0.95,0.29
8,facebook,GCN,feature,ERR,0.95,0.65
9,facebook,GAE,feature,ERR,0.96,0.65
14,facebook,GCN,structural,NDCG,0.93,0.31
20,facebook,GCN,structural,ERR,0.93,0.44
21,facebook,GAE,structural,ERR,0.93,0.45


![scale=0.5](tables/lp_ndcg.png)

![scale=0.5](tables/lp_err.png)

### Possible Extension Idea 

I am considering the creation of artificial nodes based on the concepts explored in the Computer Vision field. The concept revolves around generating new nodes by merging existing nodes from the training dataset. By taking into account the attributes of the nodes, including their relevant features, as well as the similarities observed among other nodes, my objective is to produce nodes that are located within close proximity to each individual node.